In [1]:
import pandas as pd
import pyodbc

In [2]:
aencConnection = pyodbc.connect("Driver={SQL Server};"
                                "Server=DESKTOP-8INVJ1O\SQLEXPRESS;"
                                "Database=AENC;"
                                "trusted_connection=yes")

In [ ]:
productSource = pd.read_sql("SELECT * FROM product", aencConnection)
customerSource  = pd.read_sql("SELECT * FROM customer", aencConnection)
employeeSource  = pd.read_sql("SELECT * FROM employee", aencConnection)
stateSource  = pd.read_sql("SELECT * FROM state", aencConnection)
salesOrderSource  = pd.read_sql("SELECT * FROM sales_order", aencConnection)
salesOrderItemSource  = pd.read_sql("SELECT * FROM sales_order_item", aencConnection)

In [4]:
productColumns = ['PRODUCT_id' , 'PRODUCT_name', 'PRODUCT_category' ,'PRODUCT_sub_category', 'PRODUCT_color', 'PRODUCT_prod_cost', 'PRODUCT_storage_quantity']
customerColumns = ['CUSTOMER_id', 'CUSTOMER_address', 'CUSTOMER_city', 'CUSTOMER_state', 'CUSTOMER_region', 'CUSTOMER_country', 'CUSTOMER_company_name']
employeeColumns = ['EMPLOYEE_id','EMPLOYEE_first_name','EMPLOYEE_last_name','EMPLOYEE_city','EMPLOYEE_region','EMPLOYEE_country']
dayColumns = ['DAY_date','DAY_MONTH_nr','DAY_QUARTER_nr', 'DAY_YEAR_nr']
orderDetailColumns = ['ORDER_DETAIL_NUMBER','ORDER_DETAIL_quantity', 'ORDER_DETAIL_unit_price',  'DAY_date', 'EMPLOYEE_id', 'KLANT_id', 'PRODUCT_id']

In [5]:
productDataframe = pd.DataFrame(columns=productColumns)
customerDateframe = pd.DataFrame(columns=customerColumns)
employeeDateframe = pd.DataFrame(columns=employeeColumns)
dayDateframe = pd.DataFrame(columns=dayColumns)
orderDetailDataframe = pd.DataFrame(columns=orderDetailColumns)

# Product

In [6]:
# add unit price from product to order because new product dataframe doesnt need this one but the merged fact table / star scheme does need it
selectedColumns = ['id','description' ,'name', 'Category', 'color', 'quantity', 'unit_price']
product = productSource[selectedColumns]
selectedColumns = ['prod_id','description' ,'name', 'Category', 'color', 'quantity', 'unit_price']
product.columns = selectedColumns
salesOrderItemSource = pd.merge(salesOrderItemSource, product[['prod_id', 'unit_price']],on='prod_id', how='left')
salesOrderItemSource

,id,line_id,prod_id,quantity,ship_date,unit_price
0,2001,1,300,12,15-Sep-1996 12:00:00 AM,9
1,2001,2,301,12,14-Sep-1996 12:00:00 AM,14
2,2001,3,302,12,14-Sep-1996 12:00:00 AM,14
3,2002,1,400,24,18-Sep-1996 12:00:00 AM,9
4,2002,2,401,24,18-Sep-1996 12:00:00 AM,10
...,...,...,...,...,...,...
1098,2652,2,300,10,01-Dec-1998 12:00:00 AM,9
1099,2653,1,301,0,17-Oct-1999 12:00:00 AM,14
1100,2654,1,301,15,29-Oct-1999 12:00:00 AM,14
1101,2654,2,302,3,29-Oct-1999 12:00:00 AM,14


In [7]:
selectedColumns = ['id','description' ,'name', 'Category', 'color', 'quantity']
product = productSource[selectedColumns]
columns = ['PRODUCT_id' , 'PRODUCT_name','PRODUCT_sub_category', 'PRODUCT_category', 'PRODUCT_color', 'PRODUCT_quantity']
product.columns = columns
product

,PRODUCT_id,PRODUCT_name,PRODUCT_sub_category,PRODUCT_category,PRODUCT_color,PRODUCT_quantity
0,300,Tank Top,Tee Shirt,Clothes,White,18
1,301,V-neck,Tee Shirt,Clothes,Orange,39
2,302,Crew Neck,Tee Shirt,Clothes,Black,72
3,400,Cotton Cap,Baseball Cap,Accessories,Black,92
4,401,Wool cap,Baseball Cap,Accessories,White,12
5,500,Cloth Visor,Visor,Accessories,White,36
6,501,Plastic Visor,Visor,Accessories,Black,28
7,600,Hooded Sweatshirt,Sweatshirt,Clothes,Green,39
8,601,Zipped Sweatshirt,Sweatshirt,Clothes,Blue,32
9,700,Cotton Shorts,Shorts,Clothes,Black,80


# Customer

In [8]:
customerData = pd.merge(customerSource, stateSource, left_on='state', right_on='state_id' , how='inner')

In [9]:
selectedColumns = ['id', 'address', 'city', 'state_name', 'region', 'country', 'company_name']
customerData = customerData[selectedColumns]
customerColumns = ['CUSTOMER_id', 'CUSTOMER_address', 'CUSTOMER_city', 'CUSTOMER_state', 'CUSTOMER_region', 'CUSTOMER_country', 'CUSTOMER_company_name']
customerData.columns = customerColumns
customerData

,CUSTOMER_id,CUSTOMER_address,CUSTOMER_city,CUSTOMER_state,CUSTOMER_region,CUSTOMER_country,CUSTOMER_company_name
0,101,3114 Pioneer Avenue,Rutherford,New Jersey,Eastern,USA,The Power Group
1,147,501 Third Avenue North,Wood Bridge,New Jersey,Eastern,USA,Norton Co.
2,102,1033 Whippany Road,New York,New York,Eastern,USA,AMF Corp.
3,111,14700 Prosperity Avenue,Syracuse,New York,Eastern,USA,Dynamics Inc.
4,116,39111 Wyman Street,Mamaroneck,New York,Eastern,USA,Sampson & Sons
...,...,...,...,...,...,...,...
121,331,33 West Street,New Orleans,Louisiana,South,USA,Mardi Gras Parties
122,332,67 Bronco Circle,Houston,Texas,South,USA,Bay Town Bus Co.
123,444,23 Network Way,Dallas,Texas,South,USA,Let's get Connected
124,550,6678 Sliver Road,Fargo,North Dakota,Central,USA,Flat Landers


# Employee

In [10]:
selectedColumns = ['emp_id', 'emp_fname', 'emp_lname', 'city']
employee = employeeSource[selectedColumns]
employeeColumns = ['EMPLOYEE_id','EMPLOYEE_first_name','EMPLOYEE_last_name','EMPLOYEE_city']
employee.columns = employeeColumns
employee

,EMPLOYEE_id,EMPLOYEE_first_name,EMPLOYEE_last_name,EMPLOYEE_city
0,1013,Joseph,Barker,Bedford
1,102,Fran,Whitney,Needham
2,1021,Paul,Sterling,Concord
3,1039,Shih Lin,Chao,Lexington
4,105,Matthew,Cobb,Waltham
...,...,...,...,...
70,921,Charles,Crowley,Belmont
71,930,Ann,Taylor,Atlanta
72,949,Pamela,Savarino,Long Beach
73,958,Thomas,Sisson,Houston


# Day

In [11]:
DAY_date = pd.DataFrame(columns=['DAY_date', 'DAY_MONTH_nr', 'DAY_QUARTER_nr', 'DAY_YEAR_nr'])

salesOrderSource['order_date'] = pd.to_datetime(salesOrderSource['order_date'], format='%d-%b-%Y %H:%M:%S %p')

for index, row in salesOrderSource.iterrows():
    return_date = row['order_date']

    date_values = [return_date.month, (return_date.month - 1) // 3 + 1, return_date.year]

    date_frame = pd.DataFrame([[return_date, date_values[0], date_values[1], date_values[2]]], columns=['DAY_date', 'DAY_MONTH_nr', 'DAY_QUARTER_nr', 'DAY_YEAR_nr'])
    DAY_date = pd.concat([DAY_date, date_frame], ignore_index=True)

DAY_date

,DAY_date,DAY_MONTH_nr,DAY_QUARTER_nr,DAY_YEAR_nr
0,1996-03-14 12:00:00,3,1,1996
1,1996-03-18 12:00:00,3,1,1996
2,1996-03-21 12:00:00,3,1,1996
3,1996-03-23 12:00:00,3,1,1996
4,1996-03-24 12:00:00,3,1,1996
...,...,...,...,...
645,1998-11-26 12:00:00,11,4,1998
646,1998-11-26 12:00:00,11,4,1998
647,1998-11-26 12:00:00,11,4,1998
648,1999-01-17 12:00:00,1,1,1999


# Order Detail

In [12]:
orderDetailJoin = pd.merge(salesOrderSource, salesOrderItemSource, on='id')
orderDetailJoin = pd.merge(orderDetailJoin, customerData, left_on='cust_id', right_on='CUSTOMER_id')
orderDetailJoin = pd.merge(orderDetailJoin, product, left_on='prod_id', right_on='PRODUCT_id')

In [13]:
selectedColumns = ['id','cust_id', 'order_date', 'sales_rep', 'prod_id', 'unit_price', 'quantity']
orderDetailData = orderDetailJoin[selectedColumns]
newColumnNames = ['ORDER_DETAIL_NUMBER', 'CUSTOMER_id', 'DAY_date', 'EMPLOYEE_id', 'PRODUCT_id', 'ORDER_DETAIL_unit_price', 'ORDER_DETAIL_order_quantity']
orderDetailData.columns = newColumnNames

In [14]:
orderDetailData

,ORDER_DETAIL_NUMBER,CUSTOMER_id,DAY_date,EMPLOYEE_id,PRODUCT_id,ORDER_DETAIL_unit_price,ORDER_DETAIL_order_quantity
0,2001,101,1996-03-14 12:00:00,299,300,9,12
1,2206,101,1997-10-15 12:00:00,299,300,9,12
2,2279,101,1998-01-21 12:00:00,299,300,9,12
3,2337,101,1998-03-24 12:00:00,949,300,9,12
4,2560,101,1998-09-14 12:00:00,129,300,9,24
...,...,...,...,...,...,...,...
1098,2112,165,1997-05-12 12:00:00,299,601,24,12
1099,2180,209,1997-09-09 12:00:00,902,601,24,12
1100,2227,205,1997-11-13 12:00:00,949,601,24,12
1101,2269,205,1998-01-12 12:00:00,1142,601,24,12
